In [6]:
import pandas as pd
import glob
import os

# 1. AYARLAR: Dosya İsim Şablonu
# Dosya isimlerin "...TripData" şeklinde bitiyor.
# Eğer dosyalarının uzantısı .csv ise "*TripData.csv" kullan.
# Eğer uzantı görünmüyorsa "*TripData*" da çalışır.
FILE_PATTERN = "*TripData.csv" 

print(f"📂 '{FILE_PATTERN}' şablonuna uyan dosyalar aranıyor...")

# Dosyaları listele
files = glob.glob(FILE_PATTERN)

# Listeyi kontrol et: Eğer boşsa uzantısız dene
if not files:
    print("⚠️ .csv uzantısıyla dosya bulunamadı, uzantısız aranıyor...")
    FILE_PATTERN = "*TripData*"
    files = glob.glob(FILE_PATTERN)

print(f"✅ Toplam {len(files)} adet dosya bulundu: {files[:3]} ... (ve diğerleri)")

# Koordinatları biriktireceğimiz liste
all_coords = []

# 2. DÖNGÜ: Dosyaları Tek Tek Oku (RAM Dostu)
for f in files:
    try:
        # Sadece gerekli sütunları okuyoruz (Çok hızlı)
        # Dosyanda sütun adları 'start_station_name', 'start_lat', 'start_lng' olmalı.
        df_chunk = pd.read_csv(f, usecols=['start_station_name', 'start_lat', 'start_lng'])
        
        # Bu dosyadaki benzersiz istasyonları al
        unique_stations = df_chunk.drop_duplicates('start_station_name')
        
        # Listeye ekle
        all_coords.append(unique_stations)
        
        # Bilgi ver
        print(f"   -> {f} dosyasından koordinatlar alındı.")
        
    except ValueError:
        print(f"   ⚠️ UYARI: {f} dosyasında beklenen sütunlar (start_station_name vs.) bulunamadı, geçiliyor.")
    except Exception as e:
        print(f"   ❌ HATA: {f} okunurken sorun oluştu: {e}")

# 3. ADRES DEFTERİNİ BİRLEŞTİR
if all_coords:
    print("🧩 Tüm koordinatlar birleştiriliyor...")
    master_coords = pd.concat(all_coords)
    
    # Tekrar edenleri sil (Aynı istasyon Kasım'da da Aralık'ta da vardır, tekini al)
    master_coords = master_coords.drop_duplicates('start_station_name')
    print(f"✅ Şehirdeki toplam {len(master_coords)} istasyonun koordinatı bulundu.")
    
    # 4. İSTATİSTİKLERLE EŞLEŞTİR (Denge Verisi)
    print("📊 station_stats.csv ile birleştiriliyor...")
    df_stats = pd.read_csv("station_stats.csv")
    
    # Sayı formatını temizle (Bar charttaki gibi)
    df_stats['Net_Balance'] = df_stats['Net_Balance'].astype(str).str.replace(',', '')
    df_stats['Net_Balance'] = pd.to_numeric(df_stats['Net_Balance'], errors='coerce').fillna(0)
    
    # Grupla
    df_grouped_stats = df_stats.groupby('start_station_name', as_index=False)['Net_Balance'].sum()
    
    # BİRLEŞTİR (Left Merge) -> Sadece istatistiği olanları haritaya koyacağız
    map_data = pd.merge(df_grouped_stats, master_coords, on='start_station_name', how='left')
    
    # Koordinatı bulunamayanları (örn: yeni açılan veya adı değişen) temizle
    map_data = map_data.dropna(subset=['start_lat', 'start_lng'])
    
    # 5. DOSYAYI KAYDET
    map_data.to_csv("kepler_map_data.csv", index=False)
    print(f"\n🎉 TEBRİKLER! 'kepler_map_data.csv' dosyan hazır.")
    print(f"Toplam {len(map_data)} istasyon haritalanmaya hazır.")
    
else:
    print("❌ HATA: Hiçbir dosyadan veri okunamadı. Klasörde '*TripData' isminde dosyalar olduğundan emin misin?")

📂 '*TripData.csv' şablonuna uyan dosyalar aranıyor...
✅ Toplam 12 adet dosya bulundu: ['2025_January_TripData.csv', '2025_April_TripData.csv', '2024_December_TripData.csv'] ... (ve diğerleri)
   -> 2025_January_TripData.csv dosyasından koordinatlar alındı.
   -> 2025_April_TripData.csv dosyasından koordinatlar alındı.
   -> 2024_December_TripData.csv dosyasından koordinatlar alındı.
   -> 2024_November_TripData.csv dosyasından koordinatlar alındı.
   -> 2025_February_TripData.csv dosyasından koordinatlar alındı.
   -> 2025_May_TripData.csv dosyasından koordinatlar alındı.
   -> 2025_July_TripData.csv dosyasından koordinatlar alındı.
   -> 2025_June_TripData.csv dosyasından koordinatlar alındı.
   -> 2025_October_TripData.csv dosyasından koordinatlar alındı.
   -> 2025_August_TripData.csv dosyasından koordinatlar alındı.
   -> 2025_September_TripData.csv dosyasından koordinatlar alındı.
   -> 2025_March_TripData.csv dosyasından koordinatlar alındı.
🧩 Tüm koordinatlar birleştiriliyor...


In [8]:
import pandas as pd
import glob

# 1. AYARLAR
FILE_PATTERN = "*TripData.csv" # Dosya şablonun

# 2. ÖNCE İSTATİSTİKLERİ OKU VE TEMİZLE
print("📊 station_stats.csv okunuyor...")
df_stats = pd.read_csv("station_stats.csv")

# Sayı formatı temizliği
df_stats['Net_Balance'] = df_stats['Net_Balance'].astype(str).str.replace(',', '')
df_stats['Net_Balance'] = pd.to_numeric(df_stats['Net_Balance'], errors='coerce').fillna(0)

# --- KRİTİK HAMLE 1: İsim Temizliği (Traşlama) ---
# İsimlerin başındaki/sonundaki boşlukları sil ki eşleşme hatası olmasın.
df_stats['start_station_name'] = df_stats['start_station_name'].astype(str).str.strip()

# Gruplama
df_grouped_stats = df_stats.groupby('start_station_name', as_index=False)['Net_Balance'].sum()

print(f"   -> İstatistiklerde en düşük bakiye: {df_grouped_stats['Net_Balance'].min()}")
# Burada -72k civarı bir sayı görmelisin.

# 3. KOORDİNATLARI TOPLA (Döngü)
print(f"📂 Koordinatlar '{FILE_PATTERN}' dosyalarından toplanıyor...")
files = glob.glob(FILE_PATTERN)

if not files:
    # Eğer csv bulunamazsa uzantısız ara
    files = glob.glob("*TripData*")

all_coords = []

for f in files:
    try:
        # Sadece gerekli sütunları oku
        df_chunk = pd.read_csv(f, usecols=['start_station_name', 'start_lat', 'start_lng'])
        
        # --- KRİTİK HAMLE 2: Buradaki İsimleri de Traşla ---
        df_chunk['start_station_name'] = df_chunk['start_station_name'].astype(str).str.strip()
        
        # Benzersizleri al
        unique_stations = df_chunk.drop_duplicates('start_station_name')
        all_coords.append(unique_stations)
        
    except ValueError:
        pass # Sütun yoksa geç

# 4. KOORDİNATLARI BİRLEŞTİR
if all_coords:
    master_coords = pd.concat(all_coords)
    # Tekrarları sil (Enlem/Boylam değişmez, ilkini al yeter)
    master_coords = master_coords.drop_duplicates('start_station_name')
    
    print(f"✅ Toplam {len(master_coords)} istasyonun adresi bulundu.")
    
    # 5. FİNAL BİRLEŞTİRME (MERGE)
    # how='inner' yerine 'left' kullanıyoruz ki istatistiği olup koordinatı olmayanları görelim
    map_data = pd.merge(df_grouped_stats, master_coords, on='start_station_name', how='left')
    
    # KONTROL: Koordinatı bulunamayan "önemli" istasyon var mı?
    missing = map_data[map_data['start_lat'].isnull()]
    if not missing.empty:
        print(f"⚠️ DİKKAT: {len(missing)} istasyonun koordinatı bulunamadı!")
        print("   -> Koordinatı bulunamayan en büyük istasyonlar:")
        print(missing.sort_values('Net_Balance').head(3))
    
    # Koordinatı olanları al ve kaydet
    map_data_final = map_data.dropna(subset=['start_lat', 'start_lng'])
    
    map_data_final.to_csv("kepler_map_data_fixed.csv", index=False)
    
    print("-" * 50)
    print(f"🎉 DOSYA HAZIR: 'kepler_map_data_fixed.csv'")
    print(f"Toplam İstasyon Sayısı: {len(map_data_final)}")
    
    # KANIT: En düşük bakiyeli istasyonu göster
    min_val = map_data_final['Net_Balance'].min()
    print(f"✅ Dosyadaki En Düşük Bakiye (Source): {min_val}")
    
    if min_val < -50000:
        print("🚀 BAŞARILI! -72k'lık istasyon artık dosyada mevcut.")
    else:
        print("❌ HATA: Hala yüksek değerler eksik. Dosya isimlerinde ciddi bir fark olabilir.")

else:
    print("❌ Dosya okunamadı.")

📊 station_stats.csv okunuyor...
   -> İstatistiklerde en düşük bakiye: -72696.0
📂 Koordinatlar '*TripData.csv' dosyalarından toplanıyor...
✅ Toplam 2225 istasyonun adresi bulundu.
--------------------------------------------------
🎉 DOSYA HAZIR: 'kepler_map_data_fixed.csv'
Toplam İstasyon Sayısı: 2225
✅ Dosyadaki En Düşük Bakiye (Source): -72696.0
🚀 BAŞARILI! -72k'lık istasyon artık dosyada mevcut.


In [14]:
import pandas as pd
import glob
import os

print("🎬 Animasyon için veri sıfırdan oluşturuluyor (RAM Dostu Mod)...")

# 1. DOSYA AYARLARI
FILE_PATTERN = "*TripData.csv" 
files = glob.glob(FILE_PATTERN)

if not files:
    files = glob.glob("*TripData*") # Uzantısız varsa onu dene

print(f"📂 {len(files)} adet veri dosyası taranacak...")

# Sonuçları biriktireceğimiz havuzlar
total_outflows = []
total_inflows = []

# 2. DÖNGÜ: Her dosyayı oku ve saatlik özetini çıkar
for f in files:
    try:
        print(f"   Reading: {f} ...")
        
        # Sadece gerekli sütunları oku (Hız için)
        # Bize İsim ve Zaman lazım
        cols = ['start_station_name', 'end_station_name', 'started_at', 'ended_at']
        df_chunk = pd.read_csv(f, usecols=cols)

        # --- GİDENLER (Outflows) ---
        # Saati çek (Hızlı yöntem: String kesme) "2024-01-01 14:30:00" -> "14"
        df_chunk['start_hour'] = df_chunk['started_at'].astype(str).str[11:13]
        
        # Grupla: Hangi istasyondan, hangi saatte kaç kişi çıktı?
        out_counts = df_chunk.groupby(['start_station_name', 'start_hour']).size().rename('Out_Count')
        total_outflows.append(out_counts)

        # --- GELENLER (Inflows) ---
        df_chunk['end_hour'] = df_chunk['ended_at'].astype(str).str[11:13]
        
        # Grupla: Hangi istasyona, hangi saatte kaç kişi geldi?
        in_counts = df_chunk.groupby(['end_station_name', 'end_hour']).size().rename('In_Count')
        total_inflows.append(in_counts)

        # Temizlik
        del df_chunk

    except Exception as e:
        print(f"   ⚠️ Hata ({f}): {e}")

# 3. TÜM AYLARI BİRLEŞTİR
print("🧩 Veriler birleştiriliyor...")

# Tüm gidenleri topla (İsim ve Saate göre)
all_out = pd.concat(total_outflows).groupby(level=[0, 1]).sum().reset_index()
all_out.columns = ['station_name', 'hour', 'Out_Count']

# Tüm gelenleri topla
all_in = pd.concat(total_inflows).groupby(level=[0, 1]).sum().reset_index()
all_in.columns = ['station_name', 'hour', 'In_Count']

# 4. NET DENGE HESABI (Giren - Çıkan)
# İki tabloyu birleştir
df_hourly = pd.merge(all_out, all_in, on=['station_name', 'hour'], how='outer').fillna(0)

# Net Denge
df_hourly['Net_Balance'] = df_hourly['In_Count'] - df_hourly['Out_Count']
df_hourly['Abs_Balance'] = df_hourly['Net_Balance'].abs() # Yükseklik için

print(f"✅ Saatlik istatistikler hazır! ({len(df_hourly)} satır)")

# 5. KOORDİNATLARI EKLE (Kepler İçin Şart)
print("📍 Koordinatlar ekleniyor...")
df_coords = pd.read_csv("kepler_map_data_fixed.csv") # Az önce düzelttiğimiz dosya

# İsim temizliği (Eşleşme için)
df_hourly['station_name'] = df_hourly['station_name'].astype(str).str.strip()
df_coords['start_station_name'] = df_coords['start_station_name'].astype(str).str.strip()

# Koordinatları çek
final_df = pd.merge(df_hourly, df_coords[['start_station_name', 'start_lat', 'start_lng']], 
                    left_on='station_name', right_on='start_station_name', how='inner')

# 6. KEPLER İÇİN ZAMAN FORMATI (Slider İçin)
# Sahte bir tarih ekleyerek saati Kepler'in anlayacağı formata çevir
# Örn: Saat 14 -> "2024-01-01 14:00"
final_df['real_time'] = "2024-01-01 " + final_df['hour'].astype(str).str.zfill(2) + ":00"

# Gereksiz sütunları temizle ve kaydet
columns_to_save = ['station_name', 'real_time', 'Net_Balance', 'Abs_Balance', 'start_lat', 'start_lng']
final_df = final_df[columns_to_save]

final_df.to_csv("kepler_animation_final.csv", index=False)

print("-" * 50)
print(f"🎉 FİNAL DOSYA HAZIR: 'kepler_animation_final.csv'")
print("Bu dosyayı Kepler.gl'e yükle ve Filter kısmından 'real_time' seç!")

🎬 Animasyon için veri sıfırdan oluşturuluyor (RAM Dostu Mod)...
📂 12 adet veri dosyası taranacak...
   Reading: 2025_January_TripData.csv ...
   Reading: 2025_April_TripData.csv ...
   Reading: 2024_December_TripData.csv ...
   Reading: 2024_November_TripData.csv ...
   Reading: 2025_February_TripData.csv ...
   Reading: 2025_May_TripData.csv ...
   Reading: 2025_July_TripData.csv ...
   Reading: 2025_June_TripData.csv ...
   Reading: 2025_October_TripData.csv ...
   Reading: 2025_August_TripData.csv ...
   Reading: 2025_September_TripData.csv ...
   Reading: 2025_March_TripData.csv ...
🧩 Veriler birleştiriliyor...
✅ Saatlik istatistikler hazır! (53720 satır)
📍 Koordinatlar ekleniyor...
--------------------------------------------------
🎉 FİNAL DOSYA HAZIR: 'kepler_animation_final.csv'
Bu dosyayı Kepler.gl'e yükle ve Filter kısmından 'real_time' seç!


In [19]:
import pandas as pd

print("🎬 Animasyon yumuşatılıyor (Interpolasyon)...")

# 1. MEVCUT DOSYAYI OKU
try:
    df = pd.read_csv("kepler_animation_final.csv")
    print(f"✅ Orijinal dosya okundu: {len(df)} satır.")
except FileNotFoundError:
    print("❌ HATA: 'kepler_animation_final.csv' bulunamadı. Lütfen bir önceki kodu çalıştırıp bu dosyayı oluştur.")
    exit()

# 2. ZAMAN AYARLARI
df['real_time'] = pd.to_datetime(df['real_time'])

# 3. YUMUŞATMA İŞLEMİ (DÜZELTİLEN KISIM)
# '10T' yerine '10min' kullanıyoruz.
print("⏳ Veri 10'ar dakikalık aralıklara genişletiliyor (Biraz sürebilir)...")

df_smooth = df.set_index('real_time').groupby('station_name')[['Net_Balance', 'start_lat', 'start_lng']].resample('10min').mean()

# Ara değerleri doldur (Linear Interpolation)
df_smooth['Net_Balance'] = df_smooth['Net_Balance'].interpolate(method='linear')

# Koordinatları doldur (Önceki değeri kopyala)
df_smooth['start_lat'] = df_smooth['start_lat'].ffill()
df_smooth['start_lng'] = df_smooth['start_lng'].ffill()

# İndeksi sıfırla
df_smooth = df_smooth.reset_index()

# 4. YÜKSEKLİK HESABINI GÜNCELLE
# Yumuşatılan Net_Balance'a göre yüksekliği tekrar hesapla
df_smooth['Abs_Balance'] = df_smooth['Net_Balance'].abs()

# 5. TEMİZLİK
df_smooth = df_smooth.dropna()

# 6. KAYDET
output_file = "kepler_animation_smooth_10min.csv"
df_smooth.to_csv(output_file, index=False)

print("-" * 50)
print(f"🎉 AKICI DOSYA HAZIR: '{output_file}'")
print(f"Yeni Satır Sayısı: {len(df_smooth)}")
print("Bu dosyayı Kepler.gl'e yükleyip 'real_time' filtresi eklediğinde akıcı bir geçiş göreceksin! 🚀")

🎬 Animasyon yumuşatılıyor (Interpolasyon)...
✅ Orijinal dosya okundu: 53175 satır.
⏳ Veri 10'ar dakikalık aralıklara genişletiliyor (Biraz sürebilir)...
--------------------------------------------------
🎉 AKICI DOSYA HAZIR: 'kepler_animation_smooth_10min.csv'
Yeni Satır Sayısı: 308321
Bu dosyayı Kepler.gl'e yükleyip 'real_time' filtresi eklediğinde akıcı bir geçiş göreceksin! 🚀


In [22]:
import pandas as pd
import numpy as np

print("🏗️ İstasyonlar 3D Binalara dönüştürülüyor...")

# 1. VERİYİ OKU
# En son oluşturduğumuz smooth (akıcı) dosyayı kullanıyoruz
df = pd.read_csv("kepler_animation_smooth_10min.csv")

# 2. POLYGON OLUŞTURMA FONKSİYONU
# Her noktanın etrafına yaklaşık 20 metrelik bir kare çizer.
# Lat/Lon dünyada derece cinsindendir. 0.0002 derece yaklaşık 20 metreye denk gelir.

def create_square_polygon(lat, lng, size=0.0002):
    # Karenin 4 köşesini hesapla
    lat_min = lat - size
    lat_max = lat + size
    lng_min = lng - size
    lng_max = lng + size
    
    # WKT (Well Known Text) formatında Polygon stringi oluştur
    # Format: POLYGON ((lng1 lat1, lng2 lat1, lng2 lat2, lng1 lat2, lng1 lat1))
    return f"POLYGON (({lng_min} {lat_min}, {lng_max} {lat_min}, {lng_max} {lat_max}, {lng_min} {lat_max}, {lng_min} {lat_min}))"

# 3. GEOMETRİ SÜTUNU EKLE
# Apply fonksiyonu ile her satır için kare oluşturuyoruz
df['Geometry'] = df.apply(lambda row: create_square_polygon(row['start_lat'], row['start_lng']), axis=1)

# 4. RENK İÇİN STRİNG AYARI (Opsiyonel ama garantidir)
# Bazen Kepler sayısal renklendirmede hata yapabilir, ama şu an gerek yok, Net_Balance kullanacağız.

# 5. KAYDET
output_file = "kepler_3d_buildings.csv"
df.to_csv(output_file, index=False)

print("-" * 50)
print(f"🎉 HAZIR: '{output_file}' dosyası oluşturuldu.")
print("Şimdi Kepler.gl'i aç ve bu dosyayı yükle. 'Polygon' katmanını kullanacağız!")

🏗️ İstasyonlar 3D Binalara dönüştürülüyor...
--------------------------------------------------
🎉 HAZIR: 'kepler_3d_buildings.csv' dosyası oluşturuldu.
Şimdi Kepler.gl'i aç ve bu dosyayı yükle. 'Polygon' katmanını kullanacağız!
